# Comparación estadística de Energía de señales EEG

In [1]:
from google.colab import drive

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.io as sio
from scipy import stats
import os
from scipy.stats import mannwhitneyu,levene, kstest, norm


drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Importar archivos .mat que contienen las señales desde el .zip

!cp /content/drive/MyDrive/Colab_Notebooks/datos_senales_datos_parkinson_cursos.zip .
!unzip datos_senales_datos_parkinson_cursos.zip


Archive:  datos_senales_datos_parkinson_cursos.zip
   creating: control/
  inflating: control/C001R_EP_reposo.mat  
  inflating: control/C002_EP_reposo.mat  
  inflating: control/C004_EP_reposo.mat  
  inflating: control/C005_EP_reposo_Repetido.mat  
  inflating: control/C006_EP_reposo.mat  
  inflating: control/C007_EP_reposo.mat  
  inflating: control/C010_EP_reposo.mat  
  inflating: control/C011_EP_reposo.mat  
  inflating: control/C012_EP_reposo.mat  
  inflating: control/C013_EP_reposo.mat  
  inflating: control/C015_EP_reposo.mat  
  inflating: control/C018_EP_reposo.mat  
  inflating: control/C019_EP_reposo.mat  
  inflating: control/C020_EP_reposo.mat  
  inflating: control/C021_EP_reposo.mat  
  inflating: control/C023_EP_reposo.mat  
  inflating: control/C024_EP_reposo.mat  
  inflating: control/C025_EP_reposo.mat  
  inflating: control/C026_EP_reposo.mat  
  inflating: control/C027_EP_reposo.mat  
  inflating: control/C028_EP_reposo.mat  
  inflating: control/C029_EP_reposo

In [3]:
directorio_actual = os.getcwd()
print(directorio_actual)

/content


In [4]:
archivos_control = os.listdir(directorio_actual+"/control")
print(f'Hay {len(archivos_control)} archivos control')
archivos_parkinson = os.listdir(directorio_actual+"/parkinson")
print(f'Hay {len(archivos_parkinson)} archivos parkinson')

Hay 36 archivos control
Hay 23 archivos parkinson


In [5]:
for archivo in archivos_control:
  path_subjecti = (directorio_actual + "/control"+"/"+archivo)
  #Cargar archivo
  data = sio.loadmat(path_subjecti)
  data = data['data']

  #Características
  print(f'\nArchivo: {archivo}')
  print(f'Tipo de variable cargada: {str(type(data))}')
  print(f'Dimensiones de los datos cargados: {str(data.shape)}')
  print(f'Número de dimensiones: {str(data.ndim)}')
  print(f'Tamaño: {str(data.size)}')
  print(f'Tamaño de memoria (bytes): {str(data.nbytes)}')


Archivo: C036_EP_reposo.mat
Tipo de variable cargada: <class 'numpy.ndarray'>
Dimensiones de los datos cargados: (8, 2000, 147)
Número de dimensiones: 3
Tamaño: 2352000
Tamaño de memoria (bytes): 18816000

Archivo: C037_EP_reposo.mat
Tipo de variable cargada: <class 'numpy.ndarray'>
Dimensiones de los datos cargados: (8, 2000, 181)
Número de dimensiones: 3
Tamaño: 2896000
Tamaño de memoria (bytes): 23168000

Archivo: C025_EP_reposo.mat
Tipo de variable cargada: <class 'numpy.ndarray'>
Dimensiones de los datos cargados: (8, 2000, 154)
Número de dimensiones: 3
Tamaño: 2464000
Tamaño de memoria (bytes): 19712000

Archivo: C042_EP_reposo.mat
Tipo de variable cargada: <class 'numpy.ndarray'>
Dimensiones de los datos cargados: (8, 2000, 167)
Número de dimensiones: 3
Tamaño: 2672000
Tamaño de memoria (bytes): 21376000

Archivo: C010_EP_reposo.mat
Tipo de variable cargada: <class 'numpy.ndarray'>
Dimensiones de los datos cargados: (8, 2000, 136)
Número de dimensiones: 3
Tamaño: 2176000
Tamaño

In [6]:
for archivo in archivos_parkinson:
  path_subjecti = (directorio_actual + "/parkinson"+"/"+archivo)
  #Cargar archivo
  data = sio.loadmat(path_subjecti)
  data = data['data']

  #Características
  print(f'\nArchivo: {archivo}')
  print(f'Tipo de variable cargada: {str(type(data))}')
  print(f'Dimensiones de los datos cargados: {str(data.shape)}')
  print(f'Número de dimensiones: {str(data.ndim)}')
  print(f'Tamaño: {str(data.size)}')
  print(f'Tamaño de memoria (bytes): {str(data.nbytes)}')


Archivo: P030_EP_reposo.mat
Tipo de variable cargada: <class 'numpy.ndarray'>
Dimensiones de los datos cargados: (8, 2000, 171)
Número de dimensiones: 3
Tamaño: 2736000
Tamaño de memoria (bytes): 21888000

Archivo: P026_EP_reposo.mat
Tipo de variable cargada: <class 'numpy.ndarray'>
Dimensiones de los datos cargados: (8, 2000, 142)
Número de dimensiones: 3
Tamaño: 2272000
Tamaño de memoria (bytes): 18176000

Archivo: P017_EP_reposo.mat
Tipo de variable cargada: <class 'numpy.ndarray'>
Dimensiones de los datos cargados: (8, 2000, 176)
Número de dimensiones: 3
Tamaño: 2816000
Tamaño de memoria (bytes): 22528000

Archivo: P013_EP_reposo.mat
Tipo de variable cargada: <class 'numpy.ndarray'>
Dimensiones de los datos cargados: (8, 2000, 143)
Número de dimensiones: 3
Tamaño: 2288000
Tamaño de memoria (bytes): 18304000

Archivo: P016_EP_reposo.mat
Tipo de variable cargada: <class 'numpy.ndarray'>
Dimensiones de los datos cargados: (8, 2000, 150)
Número de dimensiones: 3
Tamaño: 2400000
Tamaño

##**Implementación de una función que reciba una señal de múltiples canales y épocas y calcule la energía promedio de cada canal**

La energía de las señales de EEG puede conducir a diferenciar entre señales de pacientes con enfermedad de Parkinson y sanos. Se pide calcular la Energía promedio por grupo poblacional de cada canal.


In [7]:
def energiaPromedio(vector):
  canales, puntos, epocas = vector.shape

  energia_epocas = np.sum(vector**2,axis=1)

  energia_prom = np.mean(energia_epocas,axis=1)

  return energia_prom

Energía de cada canal promedidada por épocas para cada sujeto (para ambos grupos poblacionales).

In [8]:
def energia_prom_sujetos(path):
  archivos = os.listdir(path)

  #Matriz para almacenar E prom de cada sujeto
  matriz_energias = np.zeros((len(archivos),8))

  for i, archivo in enumerate(archivos):
    #Cargar archivo .mat

    data = sio.loadmat(os.path.join(path,archivo))['data']
    #print("Las llaves son "+ str(data.keys()))    #__header__: bytes, __version__:str, __globals__:lista, data:diccionario

    #Calcular E prom para canal
    energias_promedio = energiaPromedio(data)

    #Almacenar E prom
    matriz_energias[i,:] = energias_promedio

  return matriz_energias

Guardar la información de la Energía promediadad de cada canal promediadad por épocas en un DataFrame de columnas 'canal' y filas 'sujeto' con los valores de energía calculados.

In [9]:
#Directorio de los sujetos control y parkinson
path_control = os.getcwd()+"/control"
path_parkinson = os.getcwd()+'/parkinson'

#Energías prom por canal y sujeto
energia_controles = energia_prom_sujetos(path_control)
energia_parkinson = energia_prom_sujetos(path_parkinson)


In [10]:
import re
num_canales=8
# Creación de DataFrames para ambos grupos
#print(energia_controles)
dfcontrol = pd.DataFrame(energia_controles, columns=['Canal 1','Canal 2','Canal 3','Canal 4','Canal 5','Canal 6','Canal 7','Canal 8'])
dfparkinson = pd.DataFrame(energia_parkinson, columns=['Canal 1','Canal 2','Canal 3','Canal 4','Canal 5','Canal 6','Canal 7','Canal 8'])

#Agregar columna sujeto
dfcontrol['sujeto'] = archivos_control
dfparkinson['sujeto'] = archivos_parkinson

dfparkinson.set_index('sujeto',inplace=True)
dfcontrol.set_index('sujeto',inplace=True)

dfcontrol.index = dfcontrol.index.str.extract(r'(\d+)')[0].dropna()
dfparkinson.index = dfparkinson.index.str.extract(r'(\d+)')[0].dropna()

# Convertir los índices a tipo numérico (int)
dfcontrol.index = pd.to_numeric(dfcontrol.index, errors='coerce').astype(int)
dfparkinson.index = pd.to_numeric(dfparkinson.index, errors='coerce').astype(int)

dfcontrol.index.name = '# Sujeto'
dfparkinson.index.name = '# Sujeto'

# Ordenar los DataFrames por el índice en orden ascendente
dfcontrol.sort_index(inplace=True)
dfparkinson.sort_index(inplace=True)

print(dfcontrol)
print("\n")
print(dfparkinson)

               Canal 1       Canal 2       Canal 3       Canal 4  \
# Sujeto                                                           
1         21465.650358  20985.907912  22760.149588  18505.640284   
2         15966.402868  17617.810248  20804.937129  19654.400017   
4         14148.673322  18283.999666  28749.932148  14270.726911   
5         35311.301696  34916.686010  38800.429029  35427.031127   
6         18510.829979  19738.489375  20911.792748  21828.254399   
7         13180.109317  13925.217812  16218.994223  12324.883659   
10        11197.554574  10948.368805  12737.004665  10745.161921   
11        28551.124065  26204.839254  17383.998956  17244.605933   
12         9133.036290   9214.155028  11626.411811  10809.621612   
13        47166.556798  55107.798641  52286.884667  34682.656928   
15        17567.465030  21738.511853  29429.308030  28530.200793   
18        31250.507507  24222.776323  28298.111428  25378.777621   
19        34036.502777  35276.242239  37728.0341

#**Determinar si existe diferencia estadística entre canales de cada grupo de sujetos**

Este numeral tiene como objetivo identificar los canales que entregan información diferencial entre pacientes sanos y con enfermedad de Parkinson

##**Normalidad (Kolmogorox-Smirrov)**

In [11]:
# Ho = Los datos siguen una distribución normal
# Ha = Los datos NO siguen una distribución normal

for i in range(1,num_canales+1):

  canal_control = dfcontrol[f'Canal {i}']
  canal_parkinson = dfparkinson[f'Canal {i}']

  stat_control,p_value_control = kstest(canal_control, 'norm')
  stat_parkinson,p_value_parkinson = kstest(canal_parkinson, 'norm')

  print(f'\nCanal {i}')
  print('Control: Estadístico = {stat_control}, p-valor = {round(p_value_control,3)}')

  if p_value_control<0.05:
    print("     Rechaza hipótesis nula: Los datos NO siguen una distribución normal.")
  else:
    print("     No se rechaza hipótesis nula: Los datos siguen una distribución normal.")


  print(f'Parkinson: Estadístico = {stat_parkinson}, p-valor = {round(p_value_parkinson,3)}')

  if p_value_parkinson<0.05:
    print("     Rechaza hipótesis nula: Los datos NO siguen una distribución normal.")
  else:
    print("     No se rechaza hipótesis nula: Los datos siguen una distribución normal.")



Canal 1
Control: Estadístico = {stat_control}, p-valor = {round(p_value_control,3)}
     Rechaza hipótesis nula: Los datos NO siguen una distribución normal.
Parkinson: Estadístico = 1.0, p-valor = 0.0
     Rechaza hipótesis nula: Los datos NO siguen una distribución normal.

Canal 2
Control: Estadístico = {stat_control}, p-valor = {round(p_value_control,3)}
     Rechaza hipótesis nula: Los datos NO siguen una distribución normal.
Parkinson: Estadístico = 1.0, p-valor = 0.0
     Rechaza hipótesis nula: Los datos NO siguen una distribución normal.

Canal 3
Control: Estadístico = {stat_control}, p-valor = {round(p_value_control,3)}
     Rechaza hipótesis nula: Los datos NO siguen una distribución normal.
Parkinson: Estadístico = 1.0, p-valor = 0.0
     Rechaza hipótesis nula: Los datos NO siguen una distribución normal.

Canal 4
Control: Estadístico = {stat_control}, p-valor = {round(p_value_control,3)}
     Rechaza hipótesis nula: Los datos NO siguen una distribución normal.
Parkinson:

##**Homocedasticidad (Levene)**


Dado que no se cumple el criterio de normalidad, se emplea el test de Levene.

In [12]:
# Ho = La varianza es igual entre diferentes grupos (homocedasticidad)
# Ha = La varianza NO es igual en los diferentes grupos (heterocedasticidad)

for i in range(1, num_canales+1):

  canal_control = dfcontrol[f'Canal {i}']
  canal_parkinson = dfparkinson[f'Canal {i}']

  stat_levene,p_value_levene = levene(canal_control, canal_parkinson)

  print(f'\nCanal {i}: Estadístico = {round(stat_levene,3)}, p-valor = {round(p_value_levene,3)}')
  if p_value_levene<0.05:
    print('Se rechaza la hipostesis nula: La varianza difiere para los diferentes grupos')
  else:
    print('No se rechaza la hipotesis nula: La varianza es igual para los diferentes grupos ')



Canal 1: Estadístico = 0.021, p-valor = 0.885
No se rechaza la hipotesis nula: La varianza es igual para los diferentes grupos 

Canal 2: Estadístico = 0.003, p-valor = 0.959
No se rechaza la hipotesis nula: La varianza es igual para los diferentes grupos 

Canal 3: Estadístico = 0.001, p-valor = 0.978
No se rechaza la hipotesis nula: La varianza es igual para los diferentes grupos 

Canal 4: Estadístico = 0.095, p-valor = 0.759
No se rechaza la hipotesis nula: La varianza es igual para los diferentes grupos 

Canal 5: Estadístico = 0.015, p-valor = 0.905
No se rechaza la hipotesis nula: La varianza es igual para los diferentes grupos 

Canal 6: Estadístico = 0.524, p-valor = 0.472
No se rechaza la hipotesis nula: La varianza es igual para los diferentes grupos 

Canal 7: Estadístico = 0.252, p-valor = 0.618
No se rechaza la hipotesis nula: La varianza es igual para los diferentes grupos 

Canal 8: Estadístico = 0.287, p-valor = 0.594
No se rechaza la hipotesis nula: La varianza es ig

Las pruebas realizadas para cada uno de los canales comparando pacientes sanos y con enfermedad de Parkinson, indican que las varianzas son homogéneas entre ambos grupos, dando a entender que todos peresentan homocedasticidad. Esto sugiere que las diferencias entre los grupos no se manifiestan directamente en términos de la varianza de las señales de EEG en los canales analizados.

**Independencia** (se asume que los grupos son independientes)

No se puede realizar la prueba t ya que no se cumplen los tres requisitos (la prueba de normalidad dice que los datos no presentan una distribución normal). Por lo tanto se realiza la **Prueba U de Mann-Whitney**

##**Prueba U de Mann-Whitney**

In [13]:
# Ho = Los grupos provienen de una misma distribución (pertecenen a una misma población)
# Ha = Hay diferencias significativas entre los grupos, provienen de distribuciones diferentes (No pertenecen a una misma población)

for i in range(1, num_canales+1):

  canal_control = dfcontrol[f'Canal {i}']
  canal_parkinson = dfparkinson[f'Canal {i}']

  stat,p_value = mannwhitneyu(canal_control, canal_parkinson)

  print(f'\nCanal {i}: Estadístico = {stat}, p-valor = {round(p_value,3)}')
  if p_value<0.05:
    print('Se rechaza la hipótesis nula: hay diferencias significativas entre los grupos.')
  else:
    print('No se rechaza la hipótesis nula: no hay diferencias significativas entre los grupos.')


Canal 1: Estadístico = 360.0, p-valor = 0.406
No se rechaza la hipótesis nula: no hay diferencias significativas entre los grupos.

Canal 2: Estadístico = 377.0, p-valor = 0.571
No se rechaza la hipótesis nula: no hay diferencias significativas entre los grupos.

Canal 3: Estadístico = 366.0, p-valor = 0.46
No se rechaza la hipótesis nula: no hay diferencias significativas entre los grupos.

Canal 4: Estadístico = 337.0, p-valor = 0.234
No se rechaza la hipótesis nula: no hay diferencias significativas entre los grupos.

Canal 5: Estadístico = 376.0, p-valor = 0.56
No se rechaza la hipótesis nula: no hay diferencias significativas entre los grupos.

Canal 6: Estadístico = 344.0, p-valor = 0.28
No se rechaza la hipótesis nula: no hay diferencias significativas entre los grupos.

Canal 7: Estadístico = 313.0, p-valor = 0.118
No se rechaza la hipótesis nula: no hay diferencias significativas entre los grupos.

Canal 8: Estadístico = 321.0, p-valor = 0.151
No se rechaza la hipótesis nula:

Los resultados de las pruebas indican que ningún canal entrega información diferencial para pacientes sanos y con enfermedad Parkinson. Esto sugiere que, al menos bajo el enfoque estadístico actual, no existen diferencias detectables en la señal de EEG entre pacientes sanos y aquellos con Parkinson. Esto puede estar asociado a que se trabajó con una muestra de tamaño pequeño, en la cual puede ser difícil detectar dicha información diferencial debido a la variabilidad que se puede presentar dentro de cada grupo, un mayor tamaño de muestra podría aumentar la visión estadística y mejorar la identificación de dichas diferencias [1], también se debe tener en cuenta que no se tiene conocimiento de la fase de la enfermedad en la que se encontraban los pacientes con Parkinson. Además, puede que las diferentes más marcadas entre estos grupos no sean evidentes a nivel de la actividad cerebral capturada superficialmente, sino que estén localizados en áreas más profundas del cerebro, o se podrían manifestar en otras propiedades no lineales como la entropía u otros dominios como el análisis espectral



## Conclusión

La falta de diferencias significativas entre el grupo de pacientes sanos y aquellos con Parkinson no necesariamente indica la ausencia de diferencias reales, sino que podría reflejar las limitaciones de las pruebas estadísticas utilizadas y de las características seleccionadas para el análisis. Esto resalta la importancia de considerar factores como el tamaño de la muestra y las características clínicas de los pacientes, especialmente en enfermedades con alta variabilidad, como el Parkinson. De esta manera, se resalta la importancia del análisis de diferentes variables estadísticas que sirven como indicadores y métricas clave en la evaluación de señales de EEG.


# **Referencias**

[1] C. Fuentelsaz Gallego. “Cálculo del tamaño de la muestra”. Dr. Erwin Chiquete | Neurología | Medicina Interna – Neuroclínica | Cascada 405, Jardines del Pedregal, Consultorio 301 | Tel. citas: 5556589060 | 5556596301. Accedido el 20 de septiembre de 2024. [En línea]. Disponible: https://neuroclinica.org/wp-content/uploads/2021/09/calculo_muestra.pdf